In [1]:
import mlflow
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

data = load_iris()
X, y = data.data, data.target
model = DecisionTreeClassifier()
model.fit(X, y)

y_pred = model.predict(X)
accuracy = accuracy_score(y, y_pred)
print(f"Model accuracy: {accuracy:.2f}")


Model accuracy: 1.00


In [2]:
mlflow.set_tracking_uri(uri="http://127.0.0.1:5000")

In [3]:
mlflow.set_experiment("MLflow Iris Experiment")

2025/05/03 20:16:35 INFO mlflow.tracking.fluent: Experiment with name 'MLflow Iris Experiment' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/994234090907559238', creation_time=1746292595175, experiment_id='994234090907559238', last_update_time=1746292595175, lifecycle_stage='active', name='MLflow Iris Experiment', tags={}>

In [4]:
from mlflow.models import infer_signature
DT_params = {
    "criterion": "gini",
    "max_depth": 3,
    "min_samples_split": 2,
    "min_samples_leaf": 1,
}

model = DecisionTreeClassifier(criterion=DT_params["criterion"],
                               max_depth=DT_params["max_depth"],
                               min_samples_split=DT_params["min_samples_split"],
                               min_samples_leaf=DT_params["min_samples_leaf"])
model.fit(X, y)
y_pred = model.predict(X)
accuracy = accuracy_score(y, y_pred)

# Start an MLflow run
with mlflow.start_run():
    # Log the hyperparameters
    mlflow.log_params(DT_params)

    # Log the loss metric
    mlflow.log_metric("accuracy", accuracy)

    # Set a tag that we can use to remind ourselves what this run was for
    mlflow.set_tag("Training Info", "Basic LR model for iris data")

    # Infer the model signature
    signature = infer_signature(X, model.predict(X))

    # Log the model
    model_info = mlflow.sklearn.log_model(
        sk_model=model,
        artifact_path="iris_model_DT",
        signature=signature,
        input_example=X,
        registered_model_name="DT_iris_model",
    )

Successfully registered model 'DT_iris_model'.
2025/05/03 20:19:18 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: DT_iris_model, version 1


🏃 View run defiant-rat-691 at: http://127.0.0.1:5000/#/experiments/994234090907559238/runs/cebce6b1efc74f9facdc52a6bef98ebb
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/994234090907559238


Created version '1' of model 'DT_iris_model'.


In [5]:
loaded_model = mlflow.pyfunc.load_model(model_info.model_uri)

In [6]:
predicted = loaded_model.predict(X)
accuracy = accuracy_score(y, predicted)
print(f"Loaded model accuracy: {accuracy:.2f}")


Loaded model accuracy: 0.97


In [ ]:
from sklearn.ensemble import RandomForestClassifier

mlflow.autolog()

rf = RandomForestClassifier(n_estimators=100, max_depth=6, max_features=3)
# MLflow triggers logging automatically upon model fitting
rf.fit(X, y)
y_pred = rf.predict(X)
accuracy = accuracy_score(y, y_pred)
print(f"Random Forest Model accuracy: {accuracy:.2f}")

# The model is automatically logged to MLflow, so we don't need to log it manually
# You can view the logged model in the MLflow UI
# mlflow ui --backend-store-uri sqlite:///mlruns.db --host

2025/05/03 20:38:54 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.
2025/05/03 20:38:55 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '33e6a25e40fa4fdab00ddc21924337ac', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current sklearn workflow


🏃 View run kindly-calf-922 at: http://127.0.0.1:5000/#/experiments/994234090907559238/runs/33e6a25e40fa4fdab00ddc21924337ac
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/994234090907559238
Random Forest Model accuracy: 0.99
